# Time Series Processing

In [1]:
import os
import pandas as pd
from fastdtw import fastdtw
from statistics import mean, pstdev
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.spatial.distance import euclidean
import pygal
import seaborn as sns
import shutil
from dtaidistance import dtw, dtw_visualisation as dtwvis
from scipy import stats
from dtaidistance import clustering
from timeit import default_timer as timer

In [2]:
working_directory = "2GB-9N"

In [3]:
def get_subdirectories(directory=""):
    subdirectories = []
    p = Path(f"./../{working_directory}/{directory}")
    for item in p.glob('*/'):
        if item.suffix not in (['.csv', '.zip']):
            subdirectories.append(directory + "/" + item.name)
    return subdirectories

In [5]:
all_directories = get_subdirectories()
nodes_directories = [x for x in all_directories if "node" in x]

data_directories = []
data_directories_groups = []
for directory in nodes_directories:
    cur_node_subdirectories = get_subdirectories(directory)
    data_directories.append(cur_node_subdirectories)

data_directories_groups = data_directories
data_directories = [item for sublist in data_directories for item in sublist]

function_names = data_directories_groups[1]
function_names = list(map(lambda x: x[8:], data_directories_groups[1]))

### All functions in `experiments_data`

In [6]:
for function in function_names:
    print(f"- {function[1:]}")

- avgNetProfitGroupedBySoldDate
- avgNetProfitGroupedBySoldDateWhereProfitNegative
- avgNetProfitGroupedBySoldDateWhereYearAfter2000
- avgWholeSaleCostGroupedBySoldDate
- countNetProfitGroupedBySoldDate
- countNetProfitGroupedBySoldDateWhereProfitNegative
- countNetProfitGroupedBySoldDateWhereYearAfter2000
- countWholeSaleCostGroupedBySoldDate
- filterCatalogSalesWhereProfitNegative
- filterCatalogSalesWhereProfitNegativeAndYearAfter2000
- filterCatalogSalesWhereYearAfter2000
- filterStoreSalesWhereProfitNegative
- filterStoreSalesWhereProfitNegativeAndYearAfter2000
- filterStoreSalesWhereYearAfter2000
- maxNetProfitGroupedBySoldDate
- maxNetProfitGroupedBySoldDateWhereProfitNegative
- maxNetProfitGroupedBySoldDateWhereYearAfter2000
- maxWholeSaleCostGroupedBySoldDate
- minNetProfitGroupedBySoldDate
- minNetProfitGroupedBySoldDateWhereProfitNegative
- minNetProfitGroupedBySoldDateWhereYearAfter2000
- minWholeSaleCostGroupedBySoldDate
- summaryNetProfitGroupedBySoldDate
- summaryWholeSa

### Aggregating labels by functions names

In [7]:
labels = pd.read_csv("./../notebooks/functions.csv")
labels = labels.groupby('function_name')['label'].apply(', '.join).reset_index()
labels

,function_name,label
0,avgNetProfitGroupedBySoldDate,aggregation
1,avgNetProfitGroupedBySoldDateWhereProfitNegative,"aggregation, filtration"
2,avgNetProfitGroupedBySoldDateWhereYearAfter2000,"aggregation, filtration, join"
3,avgWholeSaleCostGroupedBySoldDate,aggregation
4,countNetProfitGroupedBySoldDate,aggregation
5,countNetProfitGroupedBySoldDateWhereProfitNega...,"aggregation, filtration"
6,countNetProfitGroupedBySoldDateWhereYearAfter2000,"aggregation, filtration, join"
7,countWholeSaleCostGroupedBySoldDate,aggregation
8,filterCatalogSalesWhereProfitNegative,filtration
9,filterCatalogSalesWhereProfitNegativeAndYearAf...,"filtration, join"


Grouping labels by UDF type
Plot colors:
    * aggregation - blue,
    * filtration - red,
    * aggregation-filtration - green,
    * aggregation-filtration-join - purple.

In [8]:
agg_labeled = labels.loc[labels["label"] == "aggregation"]
agg_fil_labeled = labels.loc[labels["label"] == "aggregation, filtration"]
fil_labeled = labels.loc[labels["label"] == "filtration"]
fil_join_labeled = labels.loc[labels["label"] == "filtration, join"]
agg_fil_join_labeled = labels.loc[labels["label"] == "aggregation, filtration, join"]

print(f"agg: {len(agg_labeled)}")
print(f"fil: {len(fil_labeled)}")
print(f"agg-fil: {len(agg_fil_labeled)}")
print(f"filtration-join: {len(fil_join_labeled)}")
print(f"agg-fil-join: {len(agg_fil_join_labeled)}")
print(f"Max: {len(function_names)} | Suma: {len(agg_labeled)+len(fil_labeled)+len(agg_fil_labeled)+len(agg_fil_join_labeled)+len(fil_join_labeled)}")

agg: 12
fil: 2
agg-fil: 5
filtration-join: 4
agg-fil-join: 5
Max: 28 | Suma: 28


In [8]:
# Function for translating (move mean to zero) and scaling (multiply by reverse standard deviation) original data (mean from worker nodes)
def translate_scale(dataframe):
    dataframe["translated"] = dataframe['CPU'] - dataframe['CPU'].mean()
    dataframe["scaled"] = dataframe["translated"] * (1/dataframe['CPU'].std())
    return dataframe

def dtw_distance(x_labels, y_labels):
    distances = []
    for x_name in x_labels["function_name"]:
        try:
            x_data = pd.read_csv(f"./../{working_directory}/preprocessed-data/workers-mean-data/{x_name}/translated_scaled_smoothed_data.csv")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        for y_name in y_labels["function_name"]:
            try:
                y_data = pd.read_csv(f"./../{working_directory}/preprocessed-data/workers-mean-data/{y_name}/translated_scaled_smoothed_data.csv")
                distance_two_sec = dtw.distance_fast(x_data["two_sec"].to_numpy(dtype=np.double), y_data["two_sec"].to_numpy(dtype=np.double))
                distances.append(distance_two_sec)
            except OSError as e:
                print("Error: %s - %s." % (e.filename, e.strerror))
    distances.sort()
    distances = [i for i in distances if i != 0.0]
    return round(min(distances), 2), round(max(distances), 2), round(mean(distances), 2), round(pstdev(distances), 2), distances,

def group_distance_statistics(labels_X, labels_Y, name_X, name_Y, dataframe):
    min_value, max_value, mean_value, std_value , all_distances = dtw_distance(labels_X, labels_Y)
    dataframe = dataframe.append({"name": f"X:{name_X} Y:{name_Y}",
                                  "min": min_value,
                                  "max": max_value,
                                  "mean": mean_value,
                                  "std": std_value,
                                  "distances": all_distances}, ignore_index=True)
    print("\n")
    print("----------------------")
    print(f"{name_X} - {name_Y}")
    print(f"MIN: {min_value}")
    print(f"MAX: {max_value}")
    print(f"MEAN: {mean_value}")
    print(f"STD: {std_value}")
    return dataframe

## Transforming data for Dynamic Time Warping

In [ ]:
# IN dataframe: "CPU"
# OUT dataframe: "translated", "scaled", "one_sec", "two_sec", "five_sec", "ten_sec"

for name in function_names:
    file_path = f"./../{working_directory}/preprocessed-data/workers-mean-data{name}"
    original_data = pd.read_csv(f"{file_path}/mean_data.csv")
    original_data = original_data[["CPU"]]
    transformed_data = translate_scale(original_data)
    transformed_data["one_sec"] = transformed_data["scaled"].rolling(4, min_periods=1, center=True).mean()
    transformed_data["two_sec"] = transformed_data["scaled"].rolling(8, min_periods=1, center=True).mean()
    transformed_data["two_half_sec"] = transformed_data["scaled"].rolling(10, min_periods=1, center=True).mean()
    transformed_data["five_sec"] = transformed_data["scaled"].rolling(20, min_periods=1, center=True).mean()
    transformed_data["ten_sec"] = transformed_data["scaled"].rolling(40, min_periods=1, center=True).mean()
    transformed_data.to_csv(f"{file_path}/translated_scaled_smoothed_data.csv", index=False)


In [49]:
del transformed_data

NameError: name 'transformed_data' is not defined

# # Comparing udfs round-robin

In [10]:
comparison_statistics = pd.DataFrame(columns=["name", "min", "max", "mean", "std", "distances"])
comparison_statistics = group_distance_statistics(agg_labeled, agg_labeled, "aggregation", "aggregation", comparison_statistics)
comparison_statistics = group_distance_statistics(agg_labeled, fil_labeled, "aggregation", "filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_labeled, fil_labeled, "filtration", "filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(agg_labeled, agg_fil_labeled, "aggregation", "aggregation-filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_labeled, agg_fil_labeled, "filtration", "aggregation-filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(agg_fil_labeled, agg_fil_labeled, "aggregation-filtration", "aggregation-filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(agg_labeled, agg_fil_join_labeled, "aggregation", "aggregation-filtration-join", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_labeled, agg_fil_join_labeled, "filtration", "aggregation-filtration-join", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_join_labeled, agg_labeled, "filtration-join", "aggregation", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_join_labeled, fil_labeled, "filtration-join", "filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_join_labeled, agg_fil_labeled, "filtration-join", "aggregation-filtration", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_join_labeled, fil_join_labeled, "filtration-join", "filtration-join", comparison_statistics)
comparison_statistics = group_distance_statistics(fil_join_labeled, agg_fil_join_labeled, "filtration-join", "aggregation-filtration-join", comparison_statistics)

# # Saving the data  with comparison statistics to file
comparison_statistics.to_csv(f"./../{working_directory}/preprocessed-data/workers-mean-data/dtw_dtai_statistics.csv", index=False)



----------------------
aggregation - aggregation
MIN: 1.6
MAX: 7.02
MEAN: 3.36
STD: 1.13


----------------------
aggregation - filtration
MIN: 20.43
MAX: 27.0
MEAN: 23.34
STD: 2.24


----------------------
filtration - filtration
MIN: 2.71
MAX: 2.71
MEAN: 2.71
STD: 0.0


----------------------
aggregation - aggregation-filtration
MIN: 3.16
MAX: 7.04
MEAN: 4.55
STD: 0.96


----------------------
filtration - aggregation-filtration
MIN: 16.07
MAX: 20.81
MEAN: 18.55
STD: 2.02


----------------------
aggregation-filtration - aggregation-filtration
MIN: 1.68
MAX: 4.29
MEAN: 2.83
STD: 0.89


----------------------
aggregation - aggregation-filtration-join
MIN: 3.74
MAX: 7.83
MEAN: 5.0
STD: 0.96


----------------------
filtration - aggregation-filtration-join
MIN: 15.76
MAX: 21.14
MEAN: 18.39
STD: 2.18


----------------------
filtration-join - aggregation
MIN: 19.47
MAX: 28.28
MEAN: 23.57
STD: 2.36


----------------------
filtration-join - filtration
MIN: 2.21
MAX: 4.54
MEAN: 3.22
STD:

## Generating dtw statistics box plot

In [11]:
box_plot = pygal.Box(truncate_legend = 25, width = 1000, height = 1000, legend_at_bottom = True, logarithmic=True)
box_plot.title = f"Function comparison statistics {working_directory}odes"
for iteration in range(comparison_statistics.shape[0]):
    box_plot.add(comparison_statistics.iloc[iteration,0], comparison_statistics.iloc[iteration,5])
box_plot.render_to_file(f"./../{working_directory}/preprocessed-data/workers-mean-data/dtai-udf-round-robin-comparison.svg")


In [56]:
del comparison_statistics

## Generating plots for every function
    * original - red,
    * transformed - green,
    * scaled - magenta,
    * 1 sec window - yellow,
    * 2 sec window - cyan,
    * 5 sec window - black,
    * 10 sec window - blue.

In [13]:
for name in function_names:

    file_path = f"./../{working_directory}/preprocessed-data/workers-mean-data{name}"
    figures_path = f"{file_path}/dtw"
    try:
        os.mkdir(figures_path)
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
    read_data = pd.read_csv(f"{file_path}/translated_scaled_smoothed_data.csv")

    fig = plt.figure(figsize=(20,10))
    fig.patch.set_facecolor('white')
    plt.plot(read_data["CPU"], 'r-', label="original")
    plt.plot(read_data["translated"], 'g-', label="translated")
    plt.plot(read_data["scaled"], 'm-', label="scaled")
    plt.title(f"Function: {name[1:]} | Plots: original, translated, scaled", fontsize=15)
    plt.xlabel("Numer próbki", fontsize=15)
    plt.ylabel("Średnie procentowe zużycie CPU", fontsize=15)
    plt.legend()
    plt.savefig(f"{figures_path}/org_trans_scaled.png")
    plt.close(fig)

    fig = plt.figure(figsize=(7,7))
    fig.patch.set_facecolor('white')
    plt.plot(read_data["scaled"], 'm-', label="scaled")
    plt.plot(read_data["one_sec"], 'k-', label="1s window")
    # plt.title(f"Function: {name[1:]} | Plots: scaled, smoothed (window: 1s)", fontsize=15)
    plt.xlabel("Numer próbki", fontsize=10)
    plt.ylabel("Średnie procentowe zużycie CPU", fontsize=10)
    plt.legend()
    plt.savefig(f"{figures_path}/1_sec_scaled.png")
    plt.close(fig)

    fig = plt.figure(figsize=(7,7))
    fig.patch.set_facecolor('white')
    plt.plot(read_data["scaled"], 'm-', label="scaled")
    plt.plot(read_data["two_sec"], 'k-', label="2s window")
    # plt.title(f"Function: {name[1:]} | Plots: scaled, smoothed (window: 2s)", fontsize=15)
    plt.xlabel("Numer próbki", fontsize=10)
    plt.ylabel("Średnie procentowe zużycie CPU", fontsize=10)
    plt.legend()
    plt.savefig(f"{figures_path}/2_sec_scaled.png")
    plt.close(fig)

    fig = plt.figure(figsize=(7,7))
    fig.patch.set_facecolor('white')
    plt.plot(read_data["scaled"], 'm-', label="scaled")
    plt.plot(read_data["two_half_sec"], 'k-', label="2.5s window")
    # plt.title(f"Function: {name[1:]} | Plots: scaled, smoothed (window: 2.5s)", fontsize=15)
    plt.xlabel("Numer próbki", fontsize=10)
    plt.ylabel("Średnie procentowe zużycie CPU", fontsize=10)
    plt.legend()
    plt.savefig(f"{figures_path}/2_5_sec_scaled.png")
    plt.close(fig)

    fig = plt.figure(figsize=(7,7))
    fig.patch.set_facecolor('white')
    plt.plot(read_data["scaled"], 'm-', label="scaled")
    plt.plot(read_data["five_sec"], 'k-', label="5s window")
    # plt.title(f"Function: {name[1:]} | Plots: scaled, smoothed (window: 5s)", fontsize=15)
    plt.xlabel("Numer próbki", fontsize=10)
    plt.ylabel("Średnie procentowe zużycie CPU", fontsize=10)
    plt.legend()
    plt.savefig(f"{figures_path}/5_sec_scaled.png")
    plt.close(fig)

    fig = plt.figure(figsize=(7,7))
    fig.patch.set_facecolor('white')
    plt.plot(read_data["scaled"], 'm-', label="scaled")
    plt.plot(read_data["ten_sec"], 'k-', label="10s window")
    # plt.title(f"Function: {name[1:]} | Plots: scaled, smoothed (window: 10s)", fontsize=15)
    plt.xlabel("Numer próbki", fontsize=10)
    plt.ylabel("Średnie procentowe zużycie CPU", fontsize=10)
    plt.legend()
    plt.savefig(f"{figures_path}/10_sec_scaled.png")
    plt.close(fig)

Error: ./../2GB-9N/preprocessed-data/workers-mean-data/avgNetProfitGroupedBySoldDate/dtw - Nie można utworzyć pliku, który już istnieje.
Error: ./../2GB-9N/preprocessed-data/workers-mean-data/avgNetProfitGroupedBySoldDateWhereProfitNegative/dtw - Nie można utworzyć pliku, który już istnieje.
Error: ./../2GB-9N/preprocessed-data/workers-mean-data/avgNetProfitGroupedBySoldDateWhereYearAfter2000/dtw - Nie można utworzyć pliku, który już istnieje.
Error: ./../2GB-9N/preprocessed-data/workers-mean-data/avgWholeSaleCostGroupedBySoldDate/dtw - Nie można utworzyć pliku, który już istnieje.
Error: ./../2GB-9N/preprocessed-data/workers-mean-data/countNetProfitGroupedBySoldDate/dtw - Nie można utworzyć pliku, który już istnieje.
Error: ./../2GB-9N/preprocessed-data/workers-mean-data/countNetProfitGroupedBySoldDateWhereProfitNegative/dtw - Nie można utworzyć pliku, który już istnieje.
Error: ./../2GB-9N/preprocessed-data/workers-mean-data/countNetProfitGroupedBySoldDateWhereYearAfter2000/dtw - Nie

## Functions of the same type in one plot

In [20]:
def plot_udf_type(udf_type, udf_type_name):
    fig = plt.figure(figsize=(20,10))
    fig.patch.set_facecolor('white')
    for name in udf_type["function_name"]:
        try:
            file_path = f"./../{working_directory}/preprocessed-data/workers-mean-data/{name}"
            read_data = pd.read_csv(f"{file_path}/translated_scaled_smoothed_data.csv")
            print(f"{name}: done")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))

        plt.plot(read_data["five_sec"], '-', label=name)
    plt.title(f" 5s window | {udf_type_name}")
    plt.legend()
    plt.savefig(f"./../{working_directory}/preprocessed-data/workers-mean-data/{udf_type_name}.png")
    plt.close(fig)

In [22]:
plot_udf_type(agg_labeled, "aggregation")
plot_udf_type(fil_labeled, "filtration")
plot_udf_type(agg_fil_labeled, "aggregation-filtration")
plot_udf_type(fil_join_labeled, "filtration-join")
plot_udf_type(agg_fil_join_labeled, "aggregation-filtration-join")

avgNetProfitGroupedBySoldDate: done
avgWholeSaleCostGroupedBySoldDate: done
countNetProfitGroupedBySoldDate: done
countWholeSaleCostGroupedBySoldDate: done
maxNetProfitGroupedBySoldDate: done
maxWholeSaleCostGroupedBySoldDate: done
minNetProfitGroupedBySoldDate: done
minWholeSaleCostGroupedBySoldDate: done
sumNetProfitGroupedBySoldDate: done
sumWholeSaleCostGroupedBySoldDate: done
summaryNetProfitGroupedBySoldDate: done
summaryWholeSaleCostGroupedBySoldDate: done
filterCatalogSalesWhereProfitNegative: done
filterStoreSalesWhereProfitNegative: done
avgNetProfitGroupedBySoldDateWhereProfitNegative: done
countNetProfitGroupedBySoldDateWhereProfitNegative: done
maxNetProfitGroupedBySoldDateWhereProfitNegative: done
minNetProfitGroupedBySoldDateWhereProfitNegative: done
sumNetProfitGroupedBySoldDateWhereProfitNegative: done
filterCatalogSalesWhereProfitNegativeAndYearAfter2000: done
filterCatalogSalesWhereYearAfter2000: done
filterStoreSalesWhereProfitNegativeAndYearAfter2000: done
filterSt

## Comparing data from 1GB and 2GB dataset sizes
Creating `udf-vs-datasetsize` folder in working directory

Creating subfolders for each function type (`agg`, `fil`, `agg-fil`, `fil-join`, `agg-fil-join`)

Gathering data about function length (number of samples) in `function_types_duration.csv`.

In [17]:
figures_path = f"./../udf-vs-datasetsize"

try:
    os.mkdir(figures_path)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

Error: ./../udf-vs-datasetsize - Nie można utworzyć pliku, który już istnieje.


In [25]:
function_type_duration = pd.DataFrame(columns=["function_type",
                                             "size-config",
                                             "length"])

dtw_vs_dataset_size = pd.DataFrame(columns=["function_name", "distance"])

function_types = [agg_labeled["function_name"],
                  fil_labeled["function_name"],
                  agg_fil_labeled["function_name"],
                  fil_join_labeled["function_name"],
                  agg_fil_join_labeled["function_name"]]

function_directory_names = ["aggregation",
                            "filtration",
                            "agg-filtration",
                            "fil-join",
                            "agg-filtration-join"]
for index in range (0,5):
    for name in function_types[index]:
        tmp_path = f"{figures_path}/{function_directory_names[index]}"
        print(f"function: {name} - done")
        try:
            os.mkdir(tmp_path)
        except OSError as e:
            # print("Error: %s - %s." % (e.filename, e.strerror))
            x = 1
        for nodes in [9]:

            one_gb_data = pd.read_csv(f"./../1GB-{nodes}N/preprocessed-data/workers-mean-data/{name}/translated_scaled_smoothed_data.csv")
            two_gb_data = pd.read_csv(f"./../2GB-{nodes}N/preprocessed-data/workers-mean-data/{name}/translated_scaled_smoothed_data.csv")

            # Measuring function type duration (A) - if (B) comment this part
            function_type_duration = function_type_duration.append({"function_type": function_directory_names[index],
                                                                    "size-config": f"1GB-{nodes}N",
                                                                    "length":  int(one_gb_data['two_sec'].shape[0])}, ignore_index=True)

            function_type_duration = function_type_duration.append({"function_type": function_directory_names[index],
                                                                    "size-config": f"2GB-{nodes}N",
                                                                    "length":  int(two_gb_data['two_sec'].shape[0])}, ignore_index=True)

            # Measuring distance between 1GB and 2GB functions (B) - if (A) comment this part
            # dtw_vs_dataset_size = dtw_vs_dataset_size.append({"function_name": name,
            #                                                   "distance": round(dtw.distance_fast(one_gb_data["two_sec"].to_numpy(dtype=np.double),
            #                                                                                       two_gb_data["two_sec"].to_numpy(dtype=np.double),
            #                                                                                       window=20,
            #                                                                                       use_pruning=True), 2)}, ignore_index=True)

            # (A) - if (B) comment this part
            fig = plt.figure(figsize=(20,10))
            fig.patch.set_facecolor('white')
            plt.plot(one_gb_data["two_sec"], 'k-', label=f"1 GB | len:{one_gb_data['two_sec'].shape[0]}")
            plt.plot(two_gb_data["two_sec"], '-', label=f"2 GB | len:{two_gb_data['two_sec'].shape[0]}", color="olive")
            plt.xlabel("Numer próbki", fontsize=15)
            plt.ylabel("Średnie procentowe zużycie CPU", fontsize=15)
            plt.legend()
            plt.savefig(f"{tmp_path}/{name}_1GBvs2GB-{nodes}N.png")
            plt.close(fig)

# (A) - if (B) comment this part
function_type_duration.to_csv(f"{figures_path}/function_types_length.csv", index=False)

# (B) - if (A) comment this part
# dtw_vs_dataset_size.to_csv(f"{figures_path}/dtw-vs-dataset-size.csv", index=False)

function: avgNetProfitGroupedBySoldDate - done
function: avgWholeSaleCostGroupedBySoldDate - done
function: countNetProfitGroupedBySoldDate - done
function: countWholeSaleCostGroupedBySoldDate - done
function: maxNetProfitGroupedBySoldDate - done
function: maxWholeSaleCostGroupedBySoldDate - done
function: minNetProfitGroupedBySoldDate - done
function: minWholeSaleCostGroupedBySoldDate - done
function: sumNetProfitGroupedBySoldDate - done
function: sumWholeSaleCostGroupedBySoldDate - done
function: summaryNetProfitGroupedBySoldDate - done
function: summaryWholeSaleCostGroupedBySoldDate - done
function: filterCatalogSalesWhereProfitNegative - done
function: filterStoreSalesWhereProfitNegative - done
function: avgNetProfitGroupedBySoldDateWhereProfitNegative - done
function: countNetProfitGroupedBySoldDateWhereProfitNegative - done
function: maxNetProfitGroupedBySoldDateWhereProfitNegative - done
function: minNetProfitGroupedBySoldDateWhereProfitNegative - done
function: sumNetProfitGrou

Changing length column to `numeric`

Aggregating data by function type and size-configuration

Gathering mean and std

Saving aggregated data to `mean-function-types-length.csv`

In [50]:
function_type_duration["length"] = pd.to_numeric(function_type_duration["length"])

function_type_mean = function_type_duration.groupby(['function_type', 'size-config']).agg(mean_length=("length", "mean"),
                                                                                          std_length=("length", "std"))
function_type_mean = function_type_mean.round(2)

function_type_mean.to_csv(f"{figures_path}/mean-function-types-length.csv")

In [52]:
for nodes in [5, 7, 9]:
    mean_nodes_length = pd.read_csv(f"{figures_path}/mean-function-types-length.csv")
    mean_nodes_length = mean_nodes_length[(mean_nodes_length["size-config"] == f"1GB-{nodes}N") | (mean_nodes_length["size-config"] == f"2GB-{nodes}N")]
    mean_nodes_length.to_csv(f"{figures_path}/{nodes}N_mean_length.csv", index=False)


## UDF vs cluster configuration 1GB

In [65]:
figures_path = f"./../udf-vs-cluster-config"
try:
    os.mkdir(figures_path)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

figures_path_vs_cluster = f"./../udf-vs-cluster-config/2GB"
try:
    os.mkdir(figures_path_vs_cluster)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))



Error: ./../udf-vs-cluster-config - Nie można utworzyć pliku, który już istnieje.


In [68]:
function_type_duration_cluster = pd.DataFrame(columns=["function_type",
                                                       "size-config",
                                                       "length"])

In [69]:

for index in range (0,5):
    for name in function_types[index]:
        five_nodes_data = pd.read_csv(f"./../2GB-5N/preprocessed-data/workers-mean-data/{name}/translated_scaled_smoothed_data.csv")
        seven_nodes_data = pd.read_csv(f"./../2GB-7N/preprocessed-data/workers-mean-data/{name}/translated_scaled_smoothed_data.csv")
        nine_nodes_data = pd.read_csv(f"./../2GB-9N/preprocessed-data/workers-mean-data/{name}/translated_scaled_smoothed_data.csv")

        tmp_path = f"{figures_path_vs_cluster}/{function_directory_names[index]}"
        print(f"function: {name} - done")
        try:
            os.mkdir(tmp_path)
        except OSError as e:
            # print("Error: %s - %s." % (e.filename, e.strerror))
            x = 1
        function_type_duration_cluster = function_type_duration_cluster.append({"function_type": function_directory_names[index],
                                                                    "size-config": f"2GB-5N",
                                                                    "length":  int(five_nodes_data['two_sec'].shape[0])}, ignore_index=True)

        function_type_duration_cluster = function_type_duration_cluster.append({"function_type": function_directory_names[index],
                                                                    "size-config": f"2GB-7N",
                                                                    "length":  int(seven_nodes_data['two_sec'].shape[0])}, ignore_index=True)

        function_type_duration_cluster = function_type_duration_cluster.append({"function_type": function_directory_names[index],
                                                                    "size-config": f"2GB-9N",
                                                                    "length":  int(nine_nodes_data['two_sec'].shape[0])}, ignore_index=True)

        fig = plt.figure(figsize=(20,10))
        fig.patch.set_facecolor('white')
        plt.plot(five_nodes_data["two_sec"], 'k-', label=f"5 węzłów | dł.:{five_nodes_data['two_sec'].shape[0]}", color="red")
        plt.plot(seven_nodes_data["two_sec"], '-', label=f"7 węzłów | dł.:{seven_nodes_data['two_sec'].shape[0]}", color="olive")
        plt.plot(nine_nodes_data["two_sec"], '-', label=f"9 węzłów | dł.:{nine_nodes_data['two_sec'].shape[0]}", color="blue")
        # plt.title(f"Function: {name[1:]} | 1GB | Nodes: 5, 7, 9", fontsize=15)
        plt.xlabel("Numer próbki", fontsize=15)
        plt.ylabel("Średnie procentowe zużycie CPU", fontsize=15)
        plt.legend()
        plt.savefig(f"{tmp_path}/{name}.png")
        plt.close(fig)
function_type_duration_cluster.to_csv(f"{figures_path}/function_types_length.csv", index=False)

function: avgNetProfitGroupedBySoldDate - done
function: avgWholeSaleCostGroupedBySoldDate - done
function: countNetProfitGroupedBySoldDate - done
function: countWholeSaleCostGroupedBySoldDate - done
function: maxNetProfitGroupedBySoldDate - done
function: maxWholeSaleCostGroupedBySoldDate - done
function: minNetProfitGroupedBySoldDate - done
function: minWholeSaleCostGroupedBySoldDate - done
function: sumNetProfitGroupedBySoldDate - done
function: sumWholeSaleCostGroupedBySoldDate - done
function: summaryNetProfitGroupedBySoldDate - done
function: summaryWholeSaleCostGroupedBySoldDate - done
function: filterCatalogSalesWhereProfitNegative - done
function: filterStoreSalesWhereProfitNegative - done
function: avgNetProfitGroupedBySoldDateWhereProfitNegative - done
function: countNetProfitGroupedBySoldDateWhereProfitNegative - done
function: maxNetProfitGroupedBySoldDateWhereProfitNegative - done
function: minNetProfitGroupedBySoldDateWhereProfitNegative - done
function: sumNetProfitGrou

C:\Users\Hubert\AppData\Local\Temp/ipykernel_3940/3891489307.py:28: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k-" (-> color='k'). The keyword argument will take precedence.
  plt.plot(five_nodes_data["two_sec"], 'k-', label=f"5 węzłów | dł.:{five_nodes_data['two_sec'].shape[0]}", color="red")


In [70]:
function_type_duration_cluster["length"] = pd.to_numeric(function_type_duration_cluster["length"])

function_type_mean_cluster = function_type_duration_cluster.groupby(['function_type', 'size-config']).agg(mean_length=("length", "mean"),
                                                                                                          std_length=("length", "std"))
function_type_mean_cluster = function_type_mean_cluster.round(2)

function_type_mean_cluster.to_csv(f"{figures_path}/mean-function-types-length.csv")

## Clean up

In [93]:
# Deleting created files

for function_name in function_names:
    try:
        os.remove(f"./../{working_directory}/preprocessed-data/workers-mean-data{function_name}/translated_scaled_smoothed_data.csv")
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

In [94]:
for function_name in function_names:
    try:
        shutil.rmtree(f"./../{working_directory}/preprocessed-data/workers-mean-data{function_name}/dtw")
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

## Comparing different DTW packages with agg functions `avgNetProfitGroupedBySoldDate` and `avgWholeSaleCostGroupedBySoldDate`.
* dtai
* fastdtw

In [91]:
dtw_comparison = pd.DataFrame(columns=["type",
                                       "distances",
                                       "durations"])

In [82]:
series_one = pd.read_csv(f"./../2GB-9N/preprocessed-data/workers-mean-data/{agg_labeled['function_name'][0]}/translated_scaled_smoothed_data.csv")
series_two = pd.read_csv(f"./../2GB-9N/preprocessed-data/workers-mean-data/{agg_labeled['function_name'][3]}/translated_scaled_smoothed_data.csv")

print(f"Series one length: {series_one.shape[0]}")
print(f"Series two length: {series_two.shape[0]}")

Series one length: 1732
Series two length: 1740


Dtai

In [88]:
dtai_durations = []
dtai_distances = []
for iteration in range(30):
    start_dtai = timer()
    dtai_distance = dtw.distance(series_one["two_sec"], series_two["two_sec"])
    end_dtai = timer()
    print(f"{iteration} {end_dtai-start_dtai}")
    dtai_durations.append(end_dtai-start_dtai)
    dtai_distances.append(dtai_distance)
print(dtai_distances)
print(dtai_durations)

0 28.380415800000264
1 27.66166020000128
2 29.415623900000355
3 26.85681730000033
4 27.40669369999887
5 26.80497739999919
6 26.71533719999934
7 26.358528300001126
8 29.553704500000094
9 26.585896300000968
10 26.175347300000794
11 25.981407799999943
12 25.64267860000109
13 25.80749999999898
14 26.07061860000067
15 26.379649500000596
16 25.69651089999934
17 25.86894620000021
18 25.629505599999902
19 25.86495079999986
20 31.54584309999882
21 31.934932400001344
22 28.926208699998824
23 27.802554199999577
24 29.137903200000437
25 31.171985700000732
26 34.10939320000034
27 30.564726900000096
28 32.20590909999919
29 26.544843499999843
[2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6

In [93]:
dtw_comparison  = dtw_comparison.append({"type": "dtai",
                                         "distances": dtai_distances,
                                         "durations": dtai_durations},
                                        ignore_index=True)

Dtai_fast - c implementation

In [94]:
dtai_f_durations = []
dtai_f_distances = []
for iteration in range(30):
    start_dtai_f = timer()
    dtai_f_distance = dtw.distance_fast(series_one["two_sec"].to_numpy(dtype=np.double), series_two["two_sec"].to_numpy(dtype=np.double))
    end_dtai_f = timer()
    print(f"{iteration} {end_dtai_f-start_dtai_f}")
    dtai_f_durations.append(end_dtai_f-start_dtai_f)
    dtai_f_distances.append(dtai_f_distance)
print(dtai_f_distances)
print(dtai_f_durations)

0 0.025364900000568014
1 0.026127099999939674
2 0.02418500000021595
3 0.02072790000056557
4 0.020152700000835466
5 0.019802599999820814
6 0.019811199999821838
7 0.019055200000366312
8 0.01955459999953746
9 0.019209899999623303
10 0.019991500001196982
11 0.018452299998898525
12 0.019202800000130082
13 0.024634000001242384
14 0.018911400000433787
15 0.019523500001014327
16 0.022711199999321252
17 0.020310699999754434
18 0.020627899999453803
19 0.42328290000114066
20 0.02370580000024347
21 0.0189782000015839
22 0.01901209999959974
23 0.019382000000405242
24 0.0195785000014439
25 0.022003900001436705
26 0.018768899999486166
27 0.01988589999928081
28 0.018954799999846728
29 0.018935400001282687
[2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 2.6727449869192403, 

In [98]:
dtw_comparison  = dtw_comparison.append({"type": "dtai_fast",
                                         "distances": dtai_f_distances,
                                         "durations": dtai_f_durations},
                                        ignore_index=True)

FastDTW

In [95]:
fastdtw_durations = []
fastdtw_distances = []
for iteration in range(30):
    start_fastdtw = timer()
    fastdtw_distance, fastdtw_path = fastdtw(series_one["two_sec"], series_two["two_sec"], dist=euclidean)
    end_fastdtw = timer()
    print(f"{iteration} {end_fastdtw-start_fastdtw}")
    fastdtw_durations.append(end_fastdtw-start_fastdtw)
    fastdtw_distances.append(fastdtw_distance)
print(fastdtw_distances)
print(fastdtw_durations)

0 1.7950084000003699
1 1.1660730999992666
2 1.057933199999752
3 1.0520505999993475
4 1.095794200000455
5 1.0888614999985293
6 1.2580463999984204
7 1.0814057999996294
8 1.0809062000007543
9 1.0772259999994276
10 1.0882581000005302
11 1.078080499999487
12 1.05712019999919
13 1.0507240999995702
14 1.0548827999991772
15 1.0589098000000376
16 1.1367444999996223
17 1.0608317999995052
18 1.0630043999990448
19 1.0567005000011704
20 1.064796299999216
21 1.0506772000007913
22 1.0538892000004125
23 1.0567442000010487
24 1.0518247999989399
25 1.0662634000000253
26 1.0650552000006428
27 1.0734518999997817
28 1.0703933000004326
29 1.0680647999997746
[70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.99920324894167, 70.9992032489

In [96]:
dtw_comparison  = dtw_comparison.append({"type": "fastdtw",
                                         "distances": fastdtw_distances,
                                         "durations": fastdtw_durations},
                                        ignore_index=True)


In [101]:
dtw_comparison.to_csv(f"./../dtw-comparison/dtw-comparison.csv", index=False)

In [112]:
print("dtai")
print(f"mean distance: {round(mean(dtai_distances), 3)}")
print(f"std distance: {round(np.std(dtai_distances), 3)}")
print(f"mean duration: {round(mean(dtai_durations), 3)}")
print(f"std duration: {round(np.std(dtai_durations), 3)}")

print("dtai_f")
print(f"mean distance: {round(mean(dtai_f_distances), 3)}")
print(f"std distance: {round(np.std(dtai_f_distances), 3)}")
print(f"mean duration: {round(mean(dtai_f_durations), 3)}")
print(f"std duration: {round(np.std(dtai_f_durations), 3)}")

print("fastdtw")
print(f"mean distance: {round(mean(fastdtw_distances), 3)}")
print(f"std distance: {round(np.std(fastdtw_distances), 3)}")
print(f"mean duration: {round(mean(fastdtw_durations), 3)}")
print(f"std duration: {round(np.std(fastdtw_durations), 3)}")

dtai
mean distance: 2.673
std distance: 0.0
mean duration: 27.96
std duration: 2.325
dtai_f
mean distance: 2.673
std distance: 0.0
mean duration: 0.034
std duration: 0.072
fastdtw
mean distance: 70.999
std distance: 0.0
mean duration: 1.103
std duration: 0.135


## Plots classification based on dtw measure

In [19]:
series = []

for f_name in function_names:
    tmp = pd.read_csv(f"./../{working_directory}/preprocessed-data/workers-mean-data{f_name}/translated_scaled_smoothed_data.csv")
    tmp = tmp['two_sec'].to_numpy()
    series.append(tmp)
ds = dtw.distance_matrix_fast(series)

In [20]:
model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
cluster_idx = model1.fit(series)
model2 = clustering.HierarchicalTree(model1)
cluster_idx = model2.fit(series)
model3 = clustering.LinkageTree(dtw.distance_matrix_fast, {})
cluster_idx = model3.fit(series)

In [19]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))  # attention: 2 columns needed!
ax.remove()
tree_plot = model2.plot(axes=ax,
                        filename="myplot.png",
                        show_ts_label=True,
                        show_tr_label=True,
                        ts_label_margin=-40,
                        ts_height=20,
                        )

<Figure size 1440x720 with 0 Axes>

In [32]:
for idx in range(len(function_names)):
    print(f"{idx} : {function_names[idx]}")

0 : /avgNetProfitGroupedBySoldDate
1 : /avgNetProfitGroupedBySoldDateWhereProfitNegative
2 : /avgNetProfitGroupedBySoldDateWhereYearAfter2000
3 : /avgWholeSaleCostGroupedBySoldDate
4 : /countNetProfitGroupedBySoldDate
5 : /countNetProfitGroupedBySoldDateWhereProfitNegative
6 : /countNetProfitGroupedBySoldDateWhereYearAfter2000
7 : /countWholeSaleCostGroupedBySoldDate
8 : /filterCatalogSalesWhereProfitNegative
9 : /filterCatalogSalesWhereProfitNegativeAndYearAfter2000
10 : /filterCatalogSalesWhereYearAfter2000
11 : /filterStoreSalesWhereProfitNegative
12 : /filterStoreSalesWhereProfitNegativeAndYearAfter2000
13 : /filterStoreSalesWhereYearAfter2000
14 : /maxNetProfitGroupedBySoldDate
15 : /maxNetProfitGroupedBySoldDateWhereProfitNegative
16 : /maxNetProfitGroupedBySoldDateWhereYearAfter2000
17 : /maxWholeSaleCostGroupedBySoldDate
18 : /minNetProfitGroupedBySoldDate
19 : /minNetProfitGroupedBySoldDateWhereProfitNegative
20 : /minNetProfitGroupedBySoldDateWhereYearAfter2000
21 : /minWhole

DTW measures for for 3 example functions

In [17]:
# countNetProfitGroupedBySoldDate

one_data_1 = pd.read_csv(f"./../1GB-9N/preprocessed-data/workers-mean-data/countNetProfitGroupedBySoldDate/translated_scaled_smoothed_data.csv")
two_data_1= pd.read_csv(f"./../2GB-9N/preprocessed-data/workers-mean-data/countNetProfitGroupedBySoldDate/translated_scaled_smoothed_data.csv")

distance_1 = dtw.distance_fast(one_data_1["two_sec"].to_numpy(dtype=np.double), two_data_1["two_sec"].to_numpy(dtype=np.double))
distance_1

9.981053067020381

In [18]:
# maxWholeSaleCostGroupedBySoldDate

one_data_2 = pd.read_csv(f"./../1GB-9N/preprocessed-data/workers-mean-data/maxWholeSaleCostGroupedBySoldDate/translated_scaled_smoothed_data.csv")
two_data_2 = pd.read_csv(f"./../2GB-9N/preprocessed-data/workers-mean-data/maxWholeSaleCostGroupedBySoldDate/translated_scaled_smoothed_data.csv")

distance_2 = dtw.distance_fast(one_data_2["two_sec"].to_numpy(dtype=np.double), two_data_2["two_sec"].to_numpy(dtype=np.double))
distance_2

10.61807868185549

In [25]:
# summaryNetProfitGroupedBySoldDate

one_data_3 = pd.read_csv(f"./../1GB-9N/preprocessed-data/workers-mean-data/summaryNetProfitGroupedBySoldDate/translated_scaled_smoothed_data.csv")
two_data_3 = pd.read_csv(f"./../2GB-9N/preprocessed-data/workers-mean-data/summaryNetProfitGroupedBySoldDate/translated_scaled_smoothed_data.csv")

distance_3 = dtw.distance_fast(one_data_3["two_sec"].to_numpy(dtype=np.double),
                               two_data_3["two_sec"].to_numpy(dtype=np.double),
                               use_pruning=True,
                               window=10
                               )
distance_3

11.422127654265514

In [20]:
distance_4 = dtw.distance_fast(two_data_2["two_sec"].to_numpy(dtype=np.double), two_data_3["two_sec"].to_numpy(dtype=np.double))
distance_4


5.327556372911023

In [ ]:
# figures_path_barplot = f"./../udf-vs-cluster-config"
figures_path_barplot = f"./../udf-vs-datasetsize"

## Barplot with average plot length for each UDF type (cluster config)

In [ ]:
udf_lengths = pd.read_csv(f"{figures_path_barplot}/mean-function-types-length.csv")
# cluster_config = "9N"

bar_width = 0.25
fig = plt.figure(figsize=(20,10))
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 15})

bars1 = udf_lengths[(udf_lengths["size-config"] == f"2GB-5N")]["mean_length"]
bars2 = udf_lengths[(udf_lengths["size-config"] == f"2GB-7N")]["mean_length"]
bars3 = udf_lengths[(udf_lengths["size-config"] == f"2GB-9N")]["mean_length"]

r1 = np.arange(len(bars1))
r2 = [x + bar_width for x in r1]
r3 = [x + 2*bar_width for x in r1]

plt.bar(r1, bars1, color='cornflowerblue', width=bar_width, edgecolor='white', label='5 węzłów')
plt.bar(r2, bars2, color='lightcoral', width=bar_width, edgecolor='white', label='7 węzłów')
plt.bar(r3, bars3, color='lightgreen', width=bar_width, edgecolor='white', label='9 węzłów')

plt.xlabel('Typ funkcji UDF')
plt.xticks([r + bar_width for r in range(len(bars1))], ['agregacja', 'filtracja', 'agregacja-filtracja', 'filtracja-join', 'agregacjo-filtracja-join'])
plt.legend()
plt.ylabel('Średnia długość charakterystyki \n (liczba próbek)')
plt.savefig(f"{figures_path_barplot}/mean-length-2GB-5-7-9.png")
# plt.show()
plt.close(fig)


## Barplot with average plot length for each UDF type (dataset size)

In [ ]:
udf_lengths = pd.read_csv(f"{figures_path_barplot}/mean-function-types-length.csv")
cluster_config = "9N"

bar_width = 0.25
fig = plt.figure(figsize=(20,10))
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 15})

bars1 = udf_lengths[(udf_lengths["size-config"] == f"1GB-{cluster_config}")]["mean_length"]
bars2 = udf_lengths[(udf_lengths["size-config"] == f"2GB-{cluster_config}")]["mean_length"]

r1 = np.arange(len(bars1))
r2 = [x + bar_width for x in r1]

plt.bar(r1, bars1, color='cornflowerblue', width=bar_width, edgecolor='white', label='1 GB')
plt.bar(r2, bars2, color='lightcoral', width=bar_width, edgecolor='white', label='2 GB')

plt.xlabel('Typ funkcji UDF')
plt.xticks([r + bar_width for r in range(len(bars1))], ['agregacja', 'filtracja', 'agregacja-filtracja', 'filtracja-join', 'agregacja-filtracja-join'])
plt.legend()
plt.ylabel('Średnia długość charakterystyki \n (liczba próbek)')
plt.savefig(f"{figures_path_barplot}/mean-length-1vs2-GB-{cluster_config}.png")
plt.close(fig)


